In [30]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# Make sure that caffe is on the python path:
caffe_root = '../'  # this file is expected to be in {caffe_root}/examples
import sys
sys.path.insert(0, caffe_root + 'python')

import caffe

In [31]:
caffe.set_mode_cpu()
net = caffe.Net('../models/swimmers_fullconv/swimmers_fullconv_small.prototxt', 
                '../models/swimmers_fullconv/out_new_small.binaryproto', caffe.TEST)
net.reshape()

In [32]:
# find max_kernels weight blob
max_kernels = net.params['idpr_dt'][0].data
# max_kernels.reshape(1, 152, 35, 35)
print max_kernels.shape

(1, 152, 7, 7)


In [33]:
# find scale term weight blob
scale_terms = net.params['idpr_dt'][1].data
print scale_terms.shape

(152,)


In [35]:
multiplier = 0.001
kernel = np.zeros((max_kernels.shape[2], max_kernels.shape[3]))
half_size = (kernel.shape[0] - 1) /2
tmp_sum = 0

for x in xrange(-half_size, half_size+1):
    for y in xrange(-half_size, half_size+1):
        pixel = multiplier * (x**2 + y**2)
        tmp_sum += pixel
        kernel[x + half_size ,y + half_size] = pixel

print kernel #* (1 / tmp_sum)

[[ 0.018  0.013  0.01   0.009  0.01   0.013  0.018]
 [ 0.013  0.008  0.005  0.004  0.005  0.008  0.013]
 [ 0.01   0.005  0.002  0.001  0.002  0.005  0.01 ]
 [ 0.009  0.004  0.001  0.     0.001  0.004  0.009]
 [ 0.01   0.005  0.002  0.001  0.002  0.005  0.01 ]
 [ 0.013  0.008  0.005  0.004  0.005  0.008  0.013]
 [ 0.018  0.013  0.01   0.009  0.01   0.013  0.018]]


In [36]:
# write every kernel as the same!
for i in xrange(max_kernels.shape[1]):
    max_kernels[0, i] = kernel

In [37]:
# write scale terms to 1 (for now...)

for i in xrange(scale_terms.shape[0]):
    scale_terms[i] = 1

In [38]:
print max_kernels, scale_terms

[[[[ 0.018  0.013  0.01  ...,  0.01   0.013  0.018]
   [ 0.013  0.008  0.005 ...,  0.005  0.008  0.013]
   [ 0.01   0.005  0.002 ...,  0.002  0.005  0.01 ]
   ..., 
   [ 0.01   0.005  0.002 ...,  0.002  0.005  0.01 ]
   [ 0.013  0.008  0.005 ...,  0.005  0.008  0.013]
   [ 0.018  0.013  0.01  ...,  0.01   0.013  0.018]]

  [[ 0.018  0.013  0.01  ...,  0.01   0.013  0.018]
   [ 0.013  0.008  0.005 ...,  0.005  0.008  0.013]
   [ 0.01   0.005  0.002 ...,  0.002  0.005  0.01 ]
   ..., 
   [ 0.01   0.005  0.002 ...,  0.002  0.005  0.01 ]
   [ 0.013  0.008  0.005 ...,  0.005  0.008  0.013]
   [ 0.018  0.013  0.01  ...,  0.01   0.013  0.018]]

  [[ 0.018  0.013  0.01  ...,  0.01   0.013  0.018]
   [ 0.013  0.008  0.005 ...,  0.005  0.008  0.013]
   [ 0.01   0.005  0.002 ...,  0.002  0.005  0.01 ]
   ..., 
   [ 0.01   0.005  0.002 ...,  0.002  0.005  0.01 ]
   [ 0.013  0.008  0.005 ...,  0.005  0.008  0.013]
   [ 0.018  0.013  0.01  ...,  0.01   0.013  0.018]]

  ..., 
  [[ 0.018  0.013  0.01

In [39]:
# save changes to new binaryproto
net.save('../models/swimmers_fullconv/out_new_small.binaryproto')